In [1]:
from __future__ import absolute_import, print_function

import itertools as it
from meza.io import read_csv, IterStringIO
from meza import io, process, convert
from csv2ofx import utils
from csv2ofx.ofx import OFX
from operator import itemgetter


In [2]:

# Date	
# Native Amount	
# Native Currency	
# Billed Amount	
# Billed Currency
# Has Folio	
# Merchant	
# City	
# Card Type	
# Personal Card	
# Card Account
i = 0
mapping = {
     'has_header': True, 
     'is_split': False, 
     'bank': 'Bank', 
     'delimiter': ',',
     'date': itemgetter('Datum'),
     'currency': itemgetter('Fremdwährung'),
     'account': 'N26 Auto Import', 
     'amount': lambda tr: tr['Betrag (Fremdwährung)'] or tr['Betrag (EUR)'], 
     'desc': lambda tr: 'EUR charged:' + tr['Betrag (EUR)'], 
     'notes': lambda tr: 'EUR charged:' + tr['Betrag (EUR)'], 
     'payee': itemgetter('Empfänger'), 
     'id': 1
}


In [3]:
ofx = OFX(mapping)

In [4]:
records = read_csv('n26-csv-transactions.csv')

In [5]:
groups = ofx.gen_groups(records)
trxns = ofx.gen_trxns(groups)


In [6]:
cleaned_trxns = ofx.clean_trxns(trxns)
data = utils.gen_data(cleaned_trxns)
content = it.chain([ofx.header(), ofx.gen_body(data), ofx.footer()])

lines = list(IterStringIO(content))

with open('n26-ynab.ofx', "wb" ) as file:
    file.writelines(lines)

